In [ ]:
! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import time
import numpy as np
import urllib.parse
from datetime import datetime
from datetime import timedelta
from pandas.api.types import CategoricalDtype
from rdflib import Literal, Namespace, RDF, URIRef
from rdflib.namespace import FOAF, XSD
from rdflib import Graph, Namespace, RDF, RDFS, OWL
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

### Step01: Define The file paths and load Data into Pandas dataframes and Clean


In [ ]:
# Define the file paths
files = ['Data/Lab3/Lab3_DECIDE_20221201.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

barometer_dt_raw = dfs[0]


In [ ]:
barometer_dt_raw

In [ ]:
# Rename columns
barometer_dt_raw.rename(columns={
    'N° échantillon': 'Dossier',
    'Date of Sample': 'Date',
    'Sample Type': 'SampleType',
    'METH': 'DiagnosticTest',
    'TRP': 'FarmID',
    'P_multocida': 'PM',
    'M_haemolytica': 'MH',
    'H_somnus': 'HS',
    'M_bovis': 'MB',
    'BRSV': 'BRSV',
    'PI3': 'PI3',
    'Coronavirus': 'BCV'
}, inplace=True)

# Separate ADDRESS column into Postal_code and City
barometer_dt_raw[['Postal_code', 'City']] = barometer_dt_raw['ADDRESS'].str.split(n=1, expand=True)

# Convert Postal_code to numeric
barometer_dt_raw['Postal_code'] = pd.to_numeric(barometer_dt_raw['Postal_code'], errors='coerce')

# Create new columns
barometer_dt_raw['FileNumber'] = barometer_dt_raw['Dossier'].str.slice(stop=12)
barometer_dt_raw['SampleNumber'] = barometer_dt_raw['Dossier'].str.slice(start=-3)
barometer_dt_raw['Country'] = 'Belgium'
barometer_dt_raw['LabReference'] = '3'

# Map Sample_type
sample_type_mapping = {
    'BAL': 'BAL',
    'SWAB': 'Swab',
    'CARCASS': 'Autopsy'
}
barometer_dt_raw['SampleType'] = barometer_dt_raw['SampleType'].map(sample_type_mapping)

# Map Breed
breed_mapping = {
    'MEAT': 'Beef',
    'MILK': 'Dairy',
    'MXD': 'Mixed'
}
barometer_dt_raw['Breed'] = barometer_dt_raw['SPECUL'].map(breed_mapping).fillna('Unknown')

# Map Province based on Postal_code
postal_code_conditions = [
    (barometer_dt_raw['Postal_code'].between(1000, 1299)),
    (barometer_dt_raw['Postal_code'].between(1300, 1499)),
    (barometer_dt_raw['Postal_code'].between(1500, 1999)),
    (barometer_dt_raw['Postal_code'].between(3000, 3499)),
    (barometer_dt_raw['Postal_code'].between(2000, 2999)),
    (barometer_dt_raw['Postal_code'].between(3500, 3999)),
    (barometer_dt_raw['Postal_code'].between(4000, 4999)),
    (barometer_dt_raw['Postal_code'].between(5000, 5999)),
    (barometer_dt_raw['Postal_code'].between(6000, 6599)),
    (barometer_dt_raw['Postal_code'].between(7000, 7999)),
    (barometer_dt_raw['Postal_code'].between(6600, 6999)),
    (barometer_dt_raw['Postal_code'].between(8000, 8999))
]
province_choices = [
    'Brussels', 'Walloon Brabant', 'Flemish Brabant', 'Antwerp', 'Limburg', 'Limburg',
    'Liège', 'Namur', 'Hainaut', 'Hainaut', 'Luxembourg', 'West Flanders'
]
barometer_dt_raw['Province'] = pd.Series(pd.Categorical(np.select(postal_code_conditions, province_choices, default='East Flanders')))

# Select columns
barometer_dt = barometer_dt_raw[['FileNumber', 'DiagnosticTest', 'SampleNumber', 'Country', 'LabReference', 'SampleType', 'Breed', 
                                 'PM', 'MH', 'HS', 'MB', 'BRSV', 'PI3', 'BCV', 'Date', 'Postal_code', 'Province', 'FarmID']].copy()

# Drop duplicates
barometer_dt.drop_duplicates(inplace=True)

# Hash Filenumber, Samplenumber, and Farm_ID
barometer_dt['FileNumber'] = barometer_dt['FileNumber'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
barometer_dt['SampleNumber'] = barometer_dt['SampleNumber'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
barometer_dt['FarmID'] = barometer_dt['FarmID'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())




In [ ]:
barometer_dt

In [ ]:
# Floor date to 1st of month
#barometer_dt['Floored_date'] = pd.to_datetime(barometer_dt['Date']).dt.to_period('M').dt.to_timestamp()

# Add a floored_date column
barometer_dt['Floored_date'] = barometer_dt['Date'].apply(lambda x: x - pd.to_timedelta(x.day - 1, unit='d'))



In [ ]:
barometer_dt

In [ ]:
# Group and aggregate data
barometer_groupby = barometer_dt.groupby(['LabReference', 'Country', 'Breed', 'Floored_date', 'Province', 'FarmID', 'DiagnosticTest', 'SampleType']).agg({
    'PM': 'max',
    'MH': 'max',
    'HS': 'max',
    'MB': 'max',
    'BRSV': 'max',
    'PI3': 'max',
    'BCV': 'max'
})

In [ ]:
barometer_groupby

In [ ]:
# Group and aggregate data
barometer_groupby = barometer_dt.groupby(['LabReference', 'Country', 'Breed', 'Floored_date', 'Province', 'FarmID', 'DiagnosticTest', 'SampleType']).agg({
    'PM': 'max',
    'MH': 'max',
    'HS': 'max',
    'MB': 'max',
    'BRSV': 'max',
    'PI3': 'max',
    'BCV': 'max'
}).reset_index()

# Convert the data to the long format for each pathogen separately:
pathogens = ['PM', 'MH', 'HS', 'MB', 'BRSV', 'PI3', 'BCV']
dfs = []
for pathogen in pathogens:
    melted_df = pd.melt(barometer_groupby, id_vars=['LabReference', 'Country', 'Breed', 'Floored_date', 'Province', 'FarmID', 'DiagnosticTest', 'SampleType'], value_vars=[pathogen], var_name='Pathogen', value_name='Result')
    dfs.append(melted_df)

barometer = pd.concat(dfs, ignore_index=True)

# Now you have a long-format DataFrame where each row corresponds to a single pathogen


In [ ]:
barometer = barometer_long

### Step 02: Create RDF graph and namespaces 

In [ ]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [ ]:
# Record the starting time
start_time = time.time()

# Iterate through the rows of the barometer_long dataframe and create RDF triples
for index, row in barometer.iterrows():
    PathogenCode = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    Result = row["Result"]
    LabReference = row["LabReference"]
    DiagnosticTest = row["DiagnosticTest"]
    Date = row["Date"]
    

    
    # Create a unique URI for each sample based on the row index
    CattleSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab3CattleSample_{index}")
    
    # Add sample type assertion
    g.add((CattleSample_uri, RDF.type, LHO.CattleSample))
    description = "An individual representing a sample from a cattle."
    g.add((CattleSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((CattleSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing a cattle breed of mixed meat and dairy type."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
            
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((CattleSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a cattle breed of mixed meat and dairy type."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((CattleSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))

    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        encoded_country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{encoded_country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((CattleSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("The Netherland")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))

    
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        encoded_province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{encoded_province}")
        g.add((CattleSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((CattleSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
    # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((CattleSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
        
    if "LabReference" in row and row["LabReference"]:
        LabReference = row["LabReference"]
    # Create a unique URI for the LabReference based on the value
        LabReference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{LabReference}")
        g.add((CattleSample_uri, LHO.hasLabReference, LabReference_uri))
        g.add((LabReference_uri, RDF.type, LHO.LabReference))

    if "Floored_date" in row and not pd.isnull(row["Floored_date"]):  # Check for NaN or None values
        Date = str(row["Floored_date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((CattleSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    
    # Add any additional properties related to Date as needed


# Serialize the RDF graph to a file
rdf_output_file = "output/RDFoutputCattleSampleLab3.ttl"
g.serialize(rdf_output_file, format="xml")

#Serialize the RDF graph to Turtle format and print it
turtle_data = g.serialize(format="turtle")
print(turtle_data)

# Record the ending time
end_time = time.time()

In [ ]:
# Calculate the execution time
execution_time = end_time - start_time

# Print the execution time
print("Execution time: {:.2f} seconds".format(execution_time))

In [ ]:
import sys
print(sys.getsizeof(g))

In [ ]:
turtle_data = g.serialize(format="turtle")

In [ ]:
import sys
print(sys.getsizeof(turtle_data))

### Step 4: Load the RDF data and ontology into a Panda DataFrame: 

In [ ]:


# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOnto1.0.owl"
g.parse(path_to_ontology, format="xml")
    

### Step 05: Query the data from updated ontology 

In [ ]:
# Define the SPARQL query and Query the data from the updated ontology (Simple Query)
query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
   (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)
 

WHERE {
  
  {
    ?Sample rdf:type LHO:CattleSample .
    ?Sample LHO:hasPathogen ?Pathogen .
    ?Sample LHO:hasSampleType ?SampleType .
    ?Sample LHO:hasProvince ?Province .
    ?Sample LHO:hasCountry ?Country .
    ?Sample LHO:hasBreed ?Breed .
    ?Sample LHO:hasResult ?SampleResult .
    ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""
# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))
df = pd.DataFrame(data, columns=["Sample", "Pathogen", "Breed", "SampleType",  "DiagnosticTest","Province", "Country", "SampleResult"])

# Display the dataframe
df




In [ ]:
# This SPARQL query shows the data over filter if Pathogen is MB, SampleType is Autopsy, diagnostic Test is culture and Breed is Beef 

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?LivestockProductionStages), "#") AS ?StagesName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
  
  {
    ?Sample rdf:type LHO:CattleSample .
    ?Sample LHO:hasPathogen ?Pathogen .
    FILTER (?Pathogen = LHO:MB)
    ?Sample LHO:hasSampleType ?SampleType .
    FILTER (?SampleType = LHO:Autopsy)
    ?Sample LHO:hasProvince ?Province .
    ?Sample LHO:hasCountry ?Country .
    ?Sample LHO:hasBreed ?Breed .
     FILTER (?Breed = LHO:Beef)
    ?Sample LHO:hasResult ?SampleResult .
    ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
    FILTER (?DiagnosticTest = LHO:Culture)
  }
}
"""
# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))
df = pd.DataFrame(data, columns=["Sample", "Pathogen", "Breed","ProductionStages", "SampleType",  "DiagnosticTest", "Province","Country", "SampleResult"])

# Display the dataframe
df



